In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')

In [ ]:
import os
filepath_recipes = os.path.abspath("C:/Users/zhan3/Desktop/RAW_recipes.csv")
filepath_interactions = os.path.abspath("C:/Users/zhan3/Desktop/RAW_interactions.csv")

In [ ]:
df_recipe = pd.read_csv(filepath_recipes)
df_interaction = pd.read_csv(filepath_interactions)
df = pd.merge(df_recipe, df_interaction, how='inner', left_on='id', right_on='recipe_id')

In [ ]:
df_rating = df.groupby('id').mean()['rating']

In [ ]:
df_reviews = df.groupby('id').count()['review']

In [ ]:
df_new = pd.merge(df_recipe, df_rating, how='left', on='id')

In [ ]:
df_new = df_new[['id', 'name', 'tags', 'description', 'ingredients', 'rating']]
df_new.dropna(axis=0, inplace=True)
df_final = df_new.reset_index(drop=True)

In [ ]:
df_final.head()

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#### Run this cell carefully, it will take 30 minutes.

In [ ]:
# for i in range(len(df_final)):
    
#     ingredients = df_final.loc[i]['ingredients']
#     ingredients = re.sub('[^a-zA-Z]', ' ', ingredients)
#     name = df_final.loc[i]['name']
#     name = re.sub('[^a-zA-Z]', ' ', name)
#     combined = name + ingredients
#     combined = combined.split()
#     ps = PorterStemmer()
#     combined = [ps.stem(word) for word in combined if not word in set(stopwords.words('english'))]
#     combined = ' '.join(combined)
#     df_final.at[i, "combined words"] = combined    

In [ ]:
# df_final.to_csv('recipe.csv', index=False)

In [ ]:
# df_read = pd.read_csv('recipe.csv')
# df_read = pd.merge(df_read, df_reviews, how='left', on='id')
# df_words = df_read[['id', 'combined words', 'rating', 'review']]

In [ ]:
# df_words.to_csv('recipe_final.csv')

### NLP training

In [ ]:
df_words = pd.read_csv("recipe_final.csv")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Map the word column into an array
text = df_words['combined words'].to_list()

# Calculate the TF-IDF matrix

model = TfidfVectorizer()
matrix = model.fit_transform(text)

In [ ]:
def query_preprocessing(query):
    
    query = re.sub('[^a-zA-Z]', ' ', query)
    query = query.split()
    ps = PorterStemmer()
    query = [ps.stem(word) for word in query if not word in set(stopwords.words('english'))]
    query = ' '.join(query)
    
    return query

In [ ]:
def get_recipe(query):
    
    query = query_preprocessing(query)
    query_matrix = model.transform([query])
    similarity = cosine_similarity(matrix, query_matrix)
    
    similarity.flatten()
    top10 = np.argsort(similarity.flatten())[-100:]
    length = len(top10)
#     print(sorted(similarity.flatten(), reverse=True))
    for i in range(length):
        j = top10[i]
        name = df_read.loc[j]['name']
        rating = df_read.loc[j]['rating']
        print(name)
        print(rating)
#     index = np.argmax(similarity)
#     name = df_read.loc[index]['name']
    
    return name

In [ ]:
print(get_recipe('steam rice'))

# consider drop the duplicates from name column